In [1]:
import os

In [2]:
import nltk

In [3]:
import pandas as pd

In [4]:
from sklearn.metrics import confusion_matrix

In [5]:
from TextNormalizer import *

In [6]:
import math

In [7]:
import time

In [74]:
class Model():
    def __init__(self):
        self.classes = {}
        self.logprob={} 
        self.vector_of_words=[]
        self.loglikelihood={}
               
    def trainNBClassifier(self):
         
        self.getTrainingData()    
        bigdoc={}
        
        for classe in self.classes:
            text=''
            
            number_of_documents = sum(map(lambda classes:int(classes),self.classes.values())) 
            number_of_doc_in_class = self.classes[classe]
            
            self.logprob[classe] = math.log(number_of_doc_in_class/number_of_documents)
        
            self.vector_of_words = self.extractVocabulary()
            
            files=os.listdir('./dataset/train/'+classe)
            for doc in files:
                text += self.read_file("./dataset/train/"+classe+"/"+doc)
            
            bigdoc[classe]= text
            
            words_of_classe = nltk.tokenize.word_tokenize(bigdoc[classe])
            
            for word in self.vector_of_words:
                self.loglikelihood[word,classe] = (bigdoc[classe].count(word) +1 / (len(words_of_classe)+len(self.vector_of_words)))
        
        #print(self.classes)
        print(self.logprob)
        #print(self.vector_of_words)
        #print(self.loglikelihood)
                
        return (self.classes, self.logprob, self.vector_of_words, self.loglikelihood )
    
    def predict(self, file):
            
        words=[]
        probabilityPerClass={}
        doc= self.read_file(file)
        normalizer = TextNormalizer()
        text = normalizer.normalizing(doc)
        #sentences = nltk.sent_tokenize(text)        
        #for sentence in sentences:
        words = nltk.word_tokenize(text)       
        
        for classe in self.classes:
            
            probabilityPerClass[classe] = self.logprob[classe]

            for word in words:
                if word in self.vector_of_words:
                    probabilityPerClass[classe]+= self.loglikelihood[word,classe]
            #print("CLASSE: ", probabilityPerClass)
            #print(len(self.vector_of_words))
        return max(probabilityPerClass, key=probabilityPerClass.get)
    
    def testNBClassifier(self):
        #path: caminho para o dataset test
        #buscar na pasta teste todos os arquivos e posteriormente chama predict()
        
        folders=os.listdir('./dataset/test')
        y_test={}
        y_predict={}
        
        
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset/test/'+classe))
            y_test[classe]=number_of_docs            
            y_predict[classe]=0
        
        
        t_inicial = time.time()  
        
        for classe in folders:  
            files=os.listdir('./dataset/test/'+classe)
            for doc in files: 
                predict_class = self.predict('./dataset/test/'+classe+'/'+doc)
                y_predict[predict_class]+=1 
                print("PREDICAO", y_predict)
        t_final = time.time()
        
        tempo_total = t_final - t_inicial
        #print(y_test)
        print(y_predict)
        print("elapsed time: " +str(tempo_total))

            
        
    def getTrainingData(self):
        folders=os.listdir('./dataset/train')
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset/train/'+classe))
            self.classes[classe]=number_of_docs        
          
    
    def extractVocabulary(self):
        bigtext=''
        folders=os.listdir('./dataset/train')
        for classe in folders:            
            files=os.listdir('./dataset/train/'+classe)
            for doc in files:
                bigtext += self.read_file("./dataset/train/"+classe+"/"+doc)
        normalizer = TextNormalizer()
        bigtext = normalizer.normalizing(bigtext)
        sentences = nltk.sent_tokenize(bigtext)        
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            words= [w for w in words if w.isalpha()] # remove all tokens that are not alphabetic
        
        stopwords = open('stopwords.txt', 'r').read()
        word_list = [w for w in words if not w in stopwords]
           
        return set(word_list)
            
    
    def read_file(self,filename):
        with open (filename, 'r') as file:
            text = file.read()
        return text    
    
    def save(self, model):           
        f = open('NBclassifier','w')
        f.write(str(classifier))  
        f.close()
        
    def importModel(self, filename):
        model= self.read_file(filename)
        model= eval(model)
        self.classes = model[0]
        self.logprob = model[1]
        self.vector_of_words = model[2]
        self.loglikelihood = model[3]
    

In [75]:
if __name__ == "__main__":
    model = Model()  
    classifier = model.trainNBClassifier()
    model.save(classifier)   

{'neg': -0.6931471805599453, 'pos': -0.6931471805599453}


In [73]:
new_model = Model()
new_model.importModel('NBclassifier')

new_model.testNBClassifier()


PREDICAO {'neg': 0, 'pos': 1}
PREDICAO {'neg': 0, 'pos': 2}
PREDICAO {'neg': 0, 'pos': 3}
PREDICAO {'neg': 0, 'pos': 4}
PREDICAO {'neg': 0, 'pos': 5}
PREDICAO {'neg': 0, 'pos': 6}
PREDICAO {'neg': 0, 'pos': 7}
PREDICAO {'neg': 0, 'pos': 8}
PREDICAO {'neg': 0, 'pos': 9}
PREDICAO {'neg': 0, 'pos': 10}
PREDICAO {'neg': 0, 'pos': 11}
PREDICAO {'neg': 0, 'pos': 12}
PREDICAO {'neg': 0, 'pos': 13}
PREDICAO {'neg': 0, 'pos': 14}
PREDICAO {'neg': 0, 'pos': 15}
PREDICAO {'neg': 0, 'pos': 16}
PREDICAO {'neg': 0, 'pos': 17}
PREDICAO {'neg': 0, 'pos': 18}
PREDICAO {'neg': 0, 'pos': 19}
PREDICAO {'neg': 0, 'pos': 20}
PREDICAO {'neg': 0, 'pos': 21}
PREDICAO {'neg': 0, 'pos': 22}
PREDICAO {'neg': 0, 'pos': 23}
PREDICAO {'neg': 0, 'pos': 24}
PREDICAO {'neg': 0, 'pos': 25}
PREDICAO {'neg': 0, 'pos': 26}
PREDICAO {'neg': 0, 'pos': 27}
PREDICAO {'neg': 0, 'pos': 28}
PREDICAO {'neg': 0, 'pos': 29}
PREDICAO {'neg': 0, 'pos': 30}
PREDICAO {'neg': 0, 'pos': 31}
PREDICAO {'neg': 0, 'pos': 32}
PREDICAO {'neg': 